## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Требуется оценить влияние семейного положения и количества детей клиента на качество исполнения кредитных обязательств - погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования необходимы для построения модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Перед выполнением задачи необходимо выявить некорректные данные и пропуски, описать возможные причины появления таких данных и обработать их.
Также нужно найти зависимость между семейным положением, уровнем дохода и возвратом кредита в срок, а так же проанализировать как разные цели кредита влияют на его возврат в срок.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd # импортируем библиотеку pandas
data = pd.read_csv('/datasets/data.csv') # открываем датафрейм с данными о заемщиках
data.info() # просматриваем общие сведения об имеющихся данных
display(data.head(10)) # выводим первые 10 строк таблицы для изучения

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [87]:
data.isnull().sum() # проверим на наличие пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [88]:
data.describe() # проанализируем имеющиеся данные методом describe

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [89]:
print(data['children'].value_counts()) # обнаружены артефакты в виде -1 и 20 детей

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [90]:
print(data['dob_years'].value_counts().tail(11)) # обнаружены артефакты в виде 101 заемщика с 0 возрастом

0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


### Вывод

В результате анализа общей информации о данных удалось сформировать следующие выводы:
1. В столбцах 'days_employed' и 'total_income' присутствуют по 2174 пустых ячеек, принимая во внимание специфику банковской деятельности, представляется крайне маловероятным что 10% кредитной массы выдано заемщикам без дохода или трудового стажа. Возможно произошла ошибка при выгрузке данных, заполним пустые значения в данных необходимых для предмета текущего исследования.
2. Столбцы 'total_income' и 'days_employed' имеют тип данных float64, можно 'облегчить' данные преобразовав их в int64.
3. Данные в столбце education обладают разным регистром.
4. В столбцах 'children', 'dob_years' - присутствуют аномально высокие и низкие значения, которые могут быть следствием ошибки при выгрузке данных или ошибки при их вводе. Учитывая что число аномальных данных в общей массе не превышает 1% можно убрать все строки с 'артефактами', чтобы не искажать результаты исследования.
5. В столбце 'days_employed' присутствуют отрицательные значения, которых там быть не должно, а также значения, которые превышают продолжительность человеческой жизни, что ни как не может быть соотнесено с количеством дней трудового стажа. Можно предположить, что у заказчика при вводе данных присутствует некий алгоритм для расчета или обработки трудвого стажа, что при выгрузке и привело к массовому появлению анмальных значений. Поскольку в рамках текущего исследования, для ответа на поставленные вопросы, данные столбца 'days_employed' не требуются, принято решение отказаться от его дальнейшего анализа.




### Шаг 2. Предобработка данных

### Обработка пропусков

In [91]:
data['total_income'].isnull().sum()# В столбце 'total_income' обнаружено 2174 пустых ячеек

2174

In [92]:
income_median = data['total_income'].median() # Находим медианное значение в столбце 'total_income'
print(income_median) 

145017.93753253992


In [93]:
income_mean = data['total_income'].mean() # Находим среднее арифметическое значение в столбце 'total_income'
print(income_mean) 

167422.30220817294


In [94]:
data['total_income'] = data['total_income'].fillna(income_median) # Заменим все нулевые значения в столбце 'total_income' на среднее медианное 
data['total_income'].isnull().sum() # проверяем, пропуски устранены

0

### Вывод

Для столбца total_income было найдено медианное (156105.0) и среднее арифметическое (167421.8) значение всего стобца total_income. В результате, для заполнения пропусков было выбрано медианное значение, как наименее подверженное влиянию "больших" чисел и как следствие менее искажающие результаты анализа.

### Замена типа данных

In [95]:
data['total_income'] = data['total_income'].astype('int') # Заменим тип данных в столбце 'total_income' с float64 на int64
data.info() # Проверяем, тип данных изменен

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


In [96]:
print(data['children'].value_counts()) # В столбце'children' пристуствуют аномальные данные, в том числе отрицательное количество детей

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [97]:
def children_anomaly(row): # Создадим функцию, чтобы убрать аномальные значения в столбце 'children'
    children=row['children'] 
    if children <= 5: 
        if children >= 0: 
            return children
data['children']=data.apply(children_anomaly, axis=1)
print(data['children'].value_counts())

0.0    14149
1.0     4818
2.0     2055
3.0      330
4.0       41
5.0        9
Name: children, dtype: int64


### Вывод

1. Для столбца 'total_income' мы изменили тип данных, что помимо снижения размера файла позволит нам в дальнейшем работать именно с целыми числами, что всегда удобнее.
2. Для удаления из столбца 'children' аномальных значений была написана функция 'children_anomaly'. Аномальными было решено считать данные с количесвтом детей, превышающих 5 и меньше 0. Так как количество данных признаных аномальными в столбце 'children' меньше 1% от общего числа детей, их удаление не повлияет на качество последующего анализа.  

### Обработка дубликатов

In [98]:
print(data['education'].value_counts()) # В столбце 'education' обнаружены данные в разном регистре

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


In [99]:
data['education'] = data['education'].str.lower() # Приведем данные в столбце 'education' к нижнему регистру

In [100]:
print(data['education'].value_counts()) # Проверяем, все данные теперь в одном регистре и группируются вместе

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [101]:
data.duplicated().sum() # проверим данные на наличие дубликатов

71

In [102]:
data = data.drop_duplicates().reset_index(drop = True) # удалим все найденные дубликаты
data.duplicated().sum() # проверяем, дубликаты удалены

0

### Вывод

1. В имеющихся данных был обнаружен 71 строка полностью дублирующие друг друга. Учитывая внушительное количество разнообразных и непересекающихся данных в каждой строке, представляется крайне маловероятным случайное совпадение данных в строках, даже при более чем двадцатитысячной выборке. Причины возникновения дубликатов кажутся скорее техническими и могли возникнуть в следствие ошибки операторов вносивших данные. С определенной долей вероятности можно  допустить, что 71 дубликат возникли в следствие повторных обращений клиентов банка за дополнительным кредитованием, но подтвердить данную теорию на основе имеющихся данных не представляется возможным. В любом случае, дублирующие друг друга строки представляют собой ничтожный процент от общего пула данных и их удаление ни как не отразится на результатах последующего исследования.  

### Лемматизация

In [2]:
df = data.loc[:, ['children', 'family_status', 'family_status_id', 'debt', 'total_income', 'purpose']]
display(df.head(10)) # сформируем отдельную таблицу, включающую в себя лишь данные необходимые дле дальнейшего исследования

,children,family_status,family_status_id,debt,total_income,purpose
0,1,женат / замужем,0,0,253875.639453,покупка жилья
1,1,женат / замужем,0,0,112080.014102,приобретение автомобиля
2,0,женат / замужем,0,0,145885.952297,покупка жилья
3,3,женат / замужем,0,0,267628.550329,дополнительное образование
4,0,гражданский брак,1,0,158616.077870,сыграть свадьбу
5,0,гражданский брак,1,0,255763.565419,покупка жилья
6,0,женат / замужем,0,0,240525.971920,операции с жильем
7,0,женат / замужем,0,0,135823.934197,образование
8,2,гражданский брак,1,0,95856.832424,на проведение свадьбы
9,0,женат / замужем,0,0,144425.938277,покупка жилья для семьи


In [104]:
from pymystem3 import Mystem
m = Mystem()
def lemmas_data(row): # Напишем функцию для лемматизации данных в столбце 'lemmas_data'
    purpose = row['purpose']
    lemmas = m.lemmatize(purpose) 
    return lemmas
df['purpose_lemmas'] = df.apply(lemmas_data, axis = 1) # Добавим лемматизированные данные к существующей таблице в столбец 'purpose_lemmas'
print(df['purpose_lemmas'].value_counts().head(10))


[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
Name: purpose_lemmas, dtype: int64


### Вывод

В результате лемматизации имеющихся в столбце 'purpose' данных мы видим, что чаще всего в целях, указваемых для получения кредита, фигурируют такие слова как: автомобиль, свадьба, недвижимость, жилье.

### Категоризация данных

In [105]:
def category_purpose(row): # Напишем функцию для объединения лемматизированых данных в группы
    category_purpose_word=row['purpose_lemmas'] 
    if 'автомобиль' in category_purpose_word:
        return 'Покупка aвтомобиля'
    if 'образование' in category_purpose_word:
        return 'Получение образование'
    if 'свадьба' in category_purpose_word:
        return 'Деньги для свадьбы'
    return 'Деньги для покупки жилья'
df['purpose_group']=df.apply(category_purpose, axis=1) # Добавим результат группировки в таблицу в новый столбец 'purpose_group'
print(df['purpose_group'].value_counts())


Деньги для покупки жилья    10811
Покупка aвтомобиля           4306
Получение образование        4013
Деньги для свадьбы           2324
Name: purpose_group, dtype: int64


In [106]:
print(df['debt'].value_counts()) # Подсчитаем число просроченных кредитов


0    19713
1     1741
Name: debt, dtype: int64


### Вывод

В результате категоризации лемматизированных данных удалось сформировать "топ" целей указываемых при оформлении кредита, среди которых с большим отрывом лидирует именно приобретение жилья.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [107]:
print(df['children'].value_counts()) # Выведем количество детей в столбце 'children'
# Сгруппируем данные столбцов 'debt', 'children' в таблицу 'debt_children'
debt_children = df.groupby(['debt', 'children']).agg({'children':'count'}) 
print(debt_children)
print(df['debt'].value_counts())


0.0    14091
1.0     4808
2.0     2052
3.0      330
4.0       41
5.0        9
Name: children, dtype: int64
               children
debt children          
0    0.0          13028
     1.0           4364
     2.0           1858
     3.0            303
     4.0             37
     5.0              9
1    0.0           1063
     1.0            444
     2.0            194
     3.0             27
     4.0              4
0    19713
1     1741
Name: debt, dtype: int64


In [108]:
# Добавим в таблицу 'debt_children' столбец 'percentage %' отражающий процентное соотнощение между количеством детей у заемщиков
# от кредитной массы без просроченых платежей - 'debt = 0' (19730 кредитов) и кредитной массы с просроченными платежами -
# - 'debt = 1' (1741 кредитов)

debt_children['percentage %'] = (13044/19784*100, 4365/19784*100, 1858/19784*100, 303/19784*100, 37/19784*100, 9/19784*100, 1063/1741*100, 444/1741*100, 194/1741*100, 27/1741*100, 4/1741*100)
debt_children['percentage %'] = debt_children['percentage %'].astype('int')# округлим до целого числа, приведем к типу данных int
print(debt_children)

               children  percentage %
debt children                        
0    0.0          13028            65
     1.0           4364            22
     2.0           1858             9
     3.0            303             1
     4.0             37             0
     5.0              9             0
1    0.0           1063            61
     1.0            444            25
     2.0            194            11
     3.0             27             1
     4.0              4             0


### Вывод

На основе полученных данных можно сделать вывод о том, что заемщики у которых нет детей чаще возвращают кредиты в срок (65% от займов выплачиваемых в срок при 61% от просроченых займов), чем заемщики с детьми (32% от займов выплачиваемых в срок при 38% от просроченых займов). Чтобы объяснить данное явление можно предположить, что у заемщиков с детьми чаще возникают срочные незапланированые расходы, что увеличивает риск несвоевременного платежа по кредиту. 

In [109]:
### Код ревьювера ### 

# немного дополнив твой код, можно было сразу добавить количество клиентов допустивших просрочку:

debt_children_new = (df.groupby(['children'])
                       .agg({'children':'count', 'debt':'sum'})
                       .rename(columns={'children':'total'})
                       .reset_index())

debt_children_new['debt_ratio'] = debt_children_new['debt'] / debt_children_new['total'] * 100
debt_children_new.round(2).sort_values('debt_ratio')


### Код ревьювера ###

,children,total,debt,debt_ratio
5,5.0,9,0,0.00
0,0.0,14091,1063,7.54
3,3.0,330,27,8.18
1,1.0,4808,444,9.23
2,2.0,2052,194,9.45
4,4.0,41,4,9.76


In [110]:
children_stat=data.pivot_table(index = ['children'], columns = 'debt', values = 'family_status_id', aggfunc='count')

In [111]:
new_columns = ['платежи в срок','просроченные платежи']
children_stat.set_axis(new_columns, axis='columns',inplace = True)

In [126]:
children_stat['debt_ratio'] = children_stat['просроченные платежи'] / (children_stat['просроченные платежи'] + children_stat['платежи в срок']) * 100

In [127]:
print(children_stat.round(2).sort_values('debt_ratio')) #  воот имхо это прям самый наглядный вариант при котором видно
# количесво платаежей при обоих вариантах debt и все красиво сгруппировалось по количеству детей

          платежи в срок  просроченные платежи  deb_ratio  debt_ratio
children                                                             
0.0              13028.0                1063.0       7.54        7.54
3.0                303.0                  27.0       8.18        8.18
1.0               4364.0                 444.0       9.23        9.23
2.0               1858.0                 194.0       9.45        9.45
4.0                 37.0                   4.0       9.76        9.76
5.0                  9.0                   NaN        NaN         NaN


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [114]:
print(df.groupby('family_status')['family_status_id'].count()) # Сгруппируем сеймейные статусы заемщиков 
print(df.groupby('family_status_id')['family_status'].sum()) # Соотнесем семейные статусы из столбца 'family_status' и ID семейного статуса
# Сгруппируем данные столбцов 'family_status_id', 'family_status', 'debt' в таблицу 'df_family'
df_family = df.loc[:, ['family_status_id', 'debt', 'family_status']]
debt_family = df_family.groupby(['debt', 'family_status_id']).count()
print(debt_family)
print(df['debt'].value_counts())

family_status
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
гражданский брак          4151
женат / замужем          12339
Name: family_status_id, dtype: int64
family_status_id
0    женат / замужемженат / замужемженат / замужемж...
1    гражданский бракгражданский бракгражданский бр...
2    вдовец / вдовавдовец / вдовавдовец / вдовавдов...
3    в разводев разводев разводев разводев разводев...
4    Не женат / не замужемНе женат / не замужемНе ж...
Name: family_status, dtype: object
                       family_status
debt family_status_id               
0    0                         11408
     1                          3763
     2                           896
     3                          1110
     4                          2536
1    0                           931
     1                           388
     2                            63
     3                            85
     4                           274
0    19713
1     1741
N

In [115]:
# Добавим в таблицу 'debt_family' столбец 'percentage %' отражающий процентное соотношение между количеством детей у заемщиков
# от кредитной массы без просроченных платежей - 'debt = 0' (19730 кредитов) и кредитной массы с просроченными платежами -
# - 'debt = 1' (1741 кредитов)
debt_family['percentage %'] = (11413/19784*100, 3775/19784*100, 896/19784*100, 1110/19784*100, 2536/19784*100, 931/1741*100, 388/1741*100, 63/1741*100, 85/1741*100, 274/1741*100)
debt_family['percentage %'] = debt_family['percentage %'].astype('int')# округлим до целого числа
print(debt_family)


                       family_status  percentage %
debt family_status_id                             
0    0                         11408            57
     1                          3763            19
     2                           896             4
     3                          1110             5
     4                          2536            12
1    0                           931            53
     1                           388            22
     2                            63             3
     3                            85             4
     4                           274            15


In [116]:
family_stat=data.pivot_table(index = ['family_status'], columns = 'debt', values = 'family_status_id', aggfunc='count')

In [117]:
new_columns = ['платежи в срок','просроченные платежи']
family_stat.set_axis(new_columns, axis='columns',inplace = True)

In [118]:
family_stat['debt_ratio'] = family_stat['просроченные платежи'] / (family_stat['просроченные платежи'] + family_stat['платежи в срок']) * 100


In [119]:
print(family_stat)

                       платежи в срок  просроченные платежи  debt_ratio
family_status                                                          
Не женат / не замужем            2536                   274    9.750890
в разводе                        1110                    85    7.112971
вдовец / вдова                    896                    63    6.569343
гражданский брак                 3763                   388    9.347145
женат / замужем                 11408                   931    7.545182


In [120]:
family_stat.round(2).sort_values('debt_ratio')

,платежи в срок,просроченные платежи,debt_ratio
family_status,,,
вдовец / вдова,896,63,6.57
в разводе,1110,85,7.11
женат / замужем,11408,931,7.55
гражданский брак,3763,388,9.35
Не женат / не замужем,2536,274,9.75


### Вывод

На основе полученных данных можно сделать вывод, о том что женатые/замужние заемщики чаще возвращают кредиты в срок (57% от займов выплачиваемых в срок при 53% от просроченых займов), чем холостые (12% от займов выплачиваемых в срок при 15% от просроченых займов).
Данный вывод подтверждает наблюдение о том что наличие семьи увеличивает социальную ответственность человека в целом.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [121]:
def income_category(row): # Напишем функцию для категоризации заемщиков в зависимости от уровня доходов
     if row <= 50000:  # Доход меньше 50тысяч - очень низкий
       return 'very_low_inc'
     elif 50000 < row <= 100000:
       return 'low_inc' # Доход от 50 до 100 тысяч - низкий
     elif 100000 < row <= 200000:
       return 'medium_inc' # Доход от 100 до 200 тысяч - средний
     elif 200000 < row <= 400000:
       return 'high_inc' # Доход от 200 до 400 тысяч - высокий
     else:
       return 'very_high_inc' # Доход свыше 400 тысяч - очень высокий
df['income_categories'] = df['total_income'].apply(income_category)
# Применим функцию получившуюся 'income_category' к столбцам 'debt' и 'income_categories', результат сохраним в "debt_income"
debt_income = df.groupby(['debt', 'income_categories']).agg({'income_categories':'count'}) 
print(debt_income)
print(df['debt'].value_counts())

                        income_categories
debt income_categories                   
0    high_inc                        4211
     low_inc                         3760
     medium_inc                     10896
     very_high_inc                    497
     very_low_inc                     349
1    high_inc                         327
     low_inc                          331
     medium_inc                      1029
     very_high_inc                     31
     very_low_inc                      23
0    19713
1     1741
Name: debt, dtype: int64


In [122]:
# Добавим в таблицу 'debt_income' столбец 'percentage %' отражающий процентное соотношение между количеством детей у заемщиков
# от кредитной массы без просроченных платежей - 'debt = 0' (19730 кредитов) и кредитной массы с просроченными платежами -
# - 'debt = 1' (1741 кредитов)
debt_income['percentage %'] = (4211/19784*100, 3760/19784*100, 10913/19784*100, 497/19784*100, 349/19784*100, 327/1741*100, 331/1741*100, 1029/1741*100, 31/1741*100, 23/1741*100)
debt_income['percentage %'] = debt_income['percentage %'].astype('int')# округлим до целого числа
print(debt_income)


                        income_categories  percentage %
debt income_categories                                 
0    high_inc                        4211            21
     low_inc                         3760            19
     medium_inc                     10896            55
     very_high_inc                    497             2
     very_low_inc                     349             1
1    high_inc                         327            18
     low_inc                          331            19
     medium_inc                      1029            59
     very_high_inc                     31             1
     very_low_inc                      23             1


In [123]:
# Оформим полученые данные более наглядню, сформировав сводную таблицу

income_stat=df.pivot_table(index = ['income_categories'], columns = 'debt', values = 'family_status_id', aggfunc='count')
new_columns = ['платежи в срок','просроченные платежи']
income_stat.set_axis(new_columns, axis='columns',inplace = True)
income_stat['debt_ratio'] = income_stat['просроченные платежи'] / (income_stat['просроченные платежи'] + income_stat['платежи в срок']) * 100
income_stat.round(2).sort_values('debt_ratio')


,платежи в срок,просроченные платежи,debt_ratio
income_categories,,,
very_high_inc,497,31,5.87
very_low_inc,349,23,6.18
high_inc,4211,327,7.21
low_inc,3760,331,8.09
medium_inc,10896,1029,8.63


In [ ]:
income1_stat=df.pivot_table(index = ['income_categories'], columns = 'debt', values = 'family_status_id', aggfunc='count')
new_columns = ['платежи в срок','просроченные платежи']
income1_stat.set_axis(new_columns, axis='columns',inplace = True)
income1_stat['debt_ratio'] = income_stat['просроченные платежи'] / (income_stat['просроченные платежи'] + income_stat['платежи в срок']) * 100
income1_stat.round(2).sort_values('debt_ratio')

### Вывод

Из полученных данных следует, что заемщики с высоким доходом чаще возвращают кредиты в срок (21% от займов выплачиваемых в срок при 18% от просроченых займов), чем заемщики со средним уровнем дохода (55% от займов выплачиваемых в срок при 59% от просроченых займов).


- Как разные цели кредита влияют на его возврат в срок?

In [124]:
print(df['purpose_group'].value_counts())
# Сгруппируем данные столбцов 'purpose_group'  и 'debt' в таблицу 'debt_purpose_group'
debt_purpose_group = df.groupby(['debt', 'purpose_group']).agg({'purpose_group':'count'})
print(debt_purpose_group)
print(df['debt'].value_counts())

Деньги для покупки жилья    10811
Покупка aвтомобиля           4306
Получение образование        4013
Деньги для свадьбы           2324
Name: purpose_group, dtype: int64
                               purpose_group
debt purpose_group                          
0    Деньги для покупки жилья          10029
     Деньги для свадьбы                 2138
     Покупка aвтомобиля                 3903
     Получение образование              3643
1    Деньги для покупки жилья            782
     Деньги для свадьбы                  186
     Покупка aвтомобиля                  403
     Получение образование               370
0    19713
1     1741
Name: debt, dtype: int64


In [125]:
# Добавим в таблицу 'debt_purpose_group' столбец 'percentage %' отражающий процентное соотношение между количеством детей у заемщиков
# от кредитной массы без просроченных платежей - 'debt = 0' (19730 кредитов) и кредитной массы с просроченными платежами -
# - 'debt = 1' (1741 кредитов)
debt_purpose_group['percentage %'] = (10032/19784*100, 2149/19784*100, 3905/19784*100, 3644/19784*100, 782/1741*100, 186/1741*100, 403/1741*100, 370/1741*100)
debt_purpose_group['percentage %'] = debt_purpose_group['percentage %'].astype('int')
print(debt_purpose_group)

                               purpose_group  percentage %
debt purpose_group                                        
0    Деньги для покупки жилья          10029            50
     Деньги для свадьбы                 2138            10
     Покупка aвтомобиля                 3903            19
     Получение образование              3643            18
1    Деньги для покупки жилья            782            44
     Деньги для свадьбы                  186            10
     Покупка aвтомобиля                  403            23
     Получение образование               370            21


In [130]:
# Оформим полученые данные более наглядню, сформировав сводную таблицу

purpose_stat=df.pivot_table(index = ['purpose_group'], columns = 'debt', values = 'family_status_id', aggfunc='count')
new_columns = ['платежи в срок','просроченные платежи']
purpose_stat.set_axis(new_columns, axis='columns',inplace = True)
purpose_stat['debt_ratio'] = purpose_stat['просроченные платежи'] / (purpose_stat['просроченные платежи'] + purpose_stat['платежи в срок']) * 100
purpose_stat.round(2).sort_values('debt_ratio')


,платежи в срок,просроченные платежи,debt_ratio
purpose_group,,,
Деньги для покупки жилья,10029,782,7.23
Деньги для свадьбы,2138,186,8.00
Получение образование,3643,370,9.22
Покупка aвтомобиля,3903,403,9.36


### Вывод

На основе полученных данных можно сделать вывод что просроченых платежей по кредитам взятым на прибритение жилья (50% от займов выплачиваемых в срок при 44% от просроченых займов) меньше чем по другим категориям. Причиной наблюдаемого явления может являться тот факт, что в большинстве случаев, залогом по кредиту на покупку жилья становится само приобретаемое жилье, что несомненно является существенной мотивацией для заемщика к исполнению взятых на себя обязательств.

### Шаг 4. Общий вывод

В результате анализа представленных данных удалось найти ответы на поставленные вопросы, а именно:
- Заемщики у которых нет детей чаще возвращают кредиты в срок, чем заемщики с детьми.
- "Семейные" заемщики чаще возвращают кредиты в срок, чем холостые.
- Заемщики с выскоим и низкими доходами чаще возвращают кредиты в срок, чем заемщики со средним уровнем доходов.
- Кредиты взятые на приобретение недвижимости чаще возвращают в срок, чем кредиты на свадьбы, автомобили, образование.